In [ ]:
import yaml

import jax
from brax.io import html

from rl_games.torch_runner import Runner
from rl_games.envs.brax import BraxEnv

from IPython.display import HTML, IFrame, display, clear_output
import os

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [ ]:
config_name = 'rl_games/configs/brax/ppo_ant.yaml'
trained_network = 'runs/Ant_brax_16-11-38-50/nn/Ant_brax.pth' #'nn/Ant_brax.pth'

# config_name = 'rl_games/configs/brax/ppo_humanoid.yaml'
# trained_network = 'nn/Humanoid_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_ur5e.yaml'
#trained_network = 'nn/Ur5e_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_halfcheetah.yaml'
#trained_network = 'nn/Halfcheetah_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_grasp.yaml'
#trained_network = 'nn/Grasp_brax.pth'


#config_name = 'rl_games/configs/brax/ppo_reacher.yaml'
#trained_network = 'nn/Reacher_brax.pth'

In [ ]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    runner = Runner()
    runner.load(config)
    agent = runner.create_player()
    agent.restore(trained_network)

env_config = runner.params['config']['env_config']

In [ ]:
num_actors = 1
env = BraxEnv('', num_actors, **env_config)

In [8]:
qps = []
obs = env.reset()
total_reward = 0
num_steps = 0


class QP:
    def __init__(self, qp):
        self.pos = jax.numpy.squeeze(qp.pos, axis=0)
        self.rot = jax.numpy.squeeze(qp.rot, axis=0)

is_done = False
while not is_done:
    qps.append(QP(env.env._state.qp))
    act = agent.get_action(obs)
    obs, reward, is_done, info = env.step(act)
    total_reward += reward.item()
    num_steps += 1

print('Total Reward: ', total_reward)
print('Num steps: ', num_steps)

ValueError: vmap got inconsistent sizes for array axes to be mapped:
the tree of axis sizes is:
(State(qp=QP(pos=1, rot=1, vel=1, ang=1), obs=1, reward=1, done=1, metrics={'reward_contact_cost': 1, 'reward_ctrl_cost': 1, 'reward_forward': 1, 'reward_survive': 1}, info={'first_obs': 1, 'first_qp': QP(pos=1, rot=1, vel=1, ang=1), 'steps': 1, 'truncation': 1}), 8)

In [ ]:
def visualize(sys, qps):
    return HTML(html.render(sys, qps))

In [ ]:
display(visualize(env.env._env.sys, qps))